## 데이터셋

데이터는 영화 '라라랜드'의 네이버영화의 평점에 기록되어 있는 커멘트 15,603개 입니다. 

In [1]:
from pprint import pprint

with open('./data/134963_norm.txt', encoding='utf-8') as f:
    docs = [doc.strip() for doc in f]

print('n docs = %d' % len(docs))
pprint(docs[:5])

n docs = 15603
['시사회에서 보고왔습니다동화와 재즈뮤지컬의 만남 지루하지않고 재밌습니다',
 '사랑과 꿈 그 흐름의 아름다움을 음악과 영상으로 최대한 담아놓았다 배우들 연기는 두말할것없고',
 '지금껏 영화 평가 해본 적이 없는데 진짜 최고네요 색감 스토리 음악 연기 모두ㅜㅜ최고입니다',
 '방금 시사회 보고 왔어요 배우들 매력이 눈을 뗄 수가 없게 만드네요 한편의 그림 같은 장면들도 많고 음악과 춤이 눈과 귀를 사로 잡았어요 '
 '한번 더 보고 싶네요',
 '초반부터 끝까지 재미있게 잘보다가 결말에서 고국마 왕창먹음 힐링 받는 느낌들다가 막판에 기분 잡쳤습니다 마치 감독이 하고싶은 말은 '
 '너희들이 원하는 결말은 이거지 하지만 현실은 이거다 라고 말하고 싶었나보군요']


## PyCRFSuite spacing 학습데이터 만들기

In [2]:
import sys
sys.path.append('../')

from pycrfsuite_spacing import TemplateGenerator
from pycrfsuite_spacing import CharacterFeatureTransformer
from pycrfsuite_spacing import sent_to_chartags
from pycrfsuite_spacing import sent_to_xy

sent_to_chars는 띄어쓰기가 있는 문장을 글자와 띄어쓰기 태그로 변환해줍니다. 띄어쓰기/붙여쓰기의 태그를 지정할 수 있습니다. default는 space = 1, nonspace = 0으로 int 입니다. 

In [3]:
chars, tags = sent_to_chartags('이것도 너프해 보시지', space='1', nonspace='0')
print(chars)
print(tags)

이것도너프해보시지
['0', '0', '1', '0', '0', '1', '0', '0', '1']


TemplateGenerator는 앞 2 글자부터 뒤 2 글짜까지 길이가 3인 templates를 만든다는 의미입니다. 반드시 현재 글자 (index 0)가 포함된 부분만 template으로 만듭니다. 

In [4]:
TemplateGenerator(begin=-2, end=2, min_range_length=3,max_range_length=3).tolist()

[(-2, 0), (-1, 1), (0, 2)]

to_feature()는 template을 이용하여 문장에서 features를 만드는 함수입니다. CharacterFeatureTransformer의 instance는 callable 입니다. 

In [5]:
to_feature = CharacterFeatureTransformer(TemplateGenerator(begin=-2, 
                                                           end=2,
                                                           min_range_length=3,
                                                           max_range_length=3))

x, y = sent_to_xy('이것도 너프해 보시지', to_feature)
pprint(x)
print(y)

[['X[0,2]=이것도'],
 ['X[-1,1]=이것도', 'X[0,2]=것도너'],
 ['X[-2,0]=이것도', 'X[-1,1]=것도너', 'X[0,2]=도너프'],
 ['X[-2,0]=것도너', 'X[-1,1]=도너프', 'X[0,2]=너프해'],
 ['X[-2,0]=도너프', 'X[-1,1]=너프해', 'X[0,2]=프해보'],
 ['X[-2,0]=너프해', 'X[-1,1]=프해보', 'X[0,2]=해보시'],
 ['X[-2,0]=프해보', 'X[-1,1]=해보시', 'X[0,2]=보시지'],
 ['X[-2,0]=해보시', 'X[-1,1]=보시지'],
 ['X[-2,0]=보시지']]
['0', '0', '1', '0', '0', '1', '0', '0', '1']


## PyCRFSuite spacing 모델 학습하기

In [6]:
from pycrfsuite_spacing import PyCRFSuiteSpacing

correct = PyCRFSuiteSpacing(to_feature)
correct.train(docs, 'demo_model.crfsuite')

학습된 모델을 불러올 수도 있습니다. 

In [7]:
correct.load_tagger('demo_model.crfsuite')

## PyCRFSuite spacing 으로 띄어쓰기 오류 교정하기

In [8]:
correct('이건진짜좋은영화라라랜드진짜좋은영화')

'이건 진짜 좋은 영화 라라랜드 진짜 좋은 영화'